In [1]:
from oumi.core.analyze.llm_judge_analyzer import LLMJudgeAnalyzer

print("Available preset prompts:", LLMJudgeAnalyzer.list_presets())

Available preset prompts: ['instruction_quality', 'response_quality', 'conversation_coherence', 'safety', 'helpfulness', 'factuality']


In [2]:
import os
import sys


# IMPORTANT: Set these BEFORE importing torch or any ML libraries
# Disable all GPU/MPS backends to prevent crashes with IFD analyzer
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Disable CUDA
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"  # Disable MPS memory allocation
os.environ["DISABLE_MPS_COMPAT"] = "1"  # Additional MPS disable flag
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"  # Disable HuggingFace telemetry
os.environ["TRANSFORMERS_OFFLINE"] = "0"  # Allow model downloads

# Force CPU usage in PyTorch to avoid MPS crashes
import torch

# Forcefully disable MPS before anything else
torch.set_default_device("cpu")
if hasattr(torch.backends, "mps"):
    # Monkey-patch to prevent MPS usage
    original_is_available = torch.backends.mps.is_available
    torch.backends.mps.is_available = lambda: False
    print("MPS has been disabled - forcing CPU-only mode")
else:
    print("Using CPU for all computations")

print(f"PyTorch device: {torch.get_default_device()}")
print(f"PyTorch version: {torch.__version__}")

MPS has been disabled - forcing CPU-only mode
PyTorch device: cpu
PyTorch version: 2.6.0


In [3]:
from oumi.core.configs import AnalyzeConfig, SampleAnalyzerParams

llm_response_quality_preset_params = SampleAnalyzerParams(
    id="llm_judge",
    instance_id="response_quality_preset",
    params={
        # Use a preset prompt (available: instruction_quality, response_quality,
        # conversation_coherence, safety, helpfulness, factuality)
        "prompt_preset": "response_quality",
        # Inference configuration
        "inference_config": {
            "model_name": "gpt-4o-mini",  # or "gpt-4o", "claude-3-5-sonnet-20241022", etc.
            "engine": "openai",  # or "vllm", "native" for local models
            "temperature": 0.1,  # Low temperature for consistent judgments
            "max_tokens": 256,
            # For OpenAI, API key is read from OPENAI_API_KEY env var by default
            # For Anthropic, use ANTHROPIC_API_KEY env var
            # Or specify explicitly:
            # "api_key_env": "OPENAI_API_KEY",  # or "ANTHROPIC_API_KEY"
        },
        "batch_size": 10,  # Process 10 samples at a time
        "max_text_length": 4000,  # Truncate long texts
        "parse_json_response": True,  # Parse JSON from LLM response
    },
)

llm_response_quality_params = SampleAnalyzerParams(
    id="llm_judge",
    instance_id="response_quality",
    params={
        # Custom prompt for classification task evaluation
        "prompt": """Evaluate this assistant's classification response (0-10).

Context: The assistant must classify a banking query by responding with a single intent ID (0-76).

Assistant response to evaluate:
{text}

Evaluation criteria:
- Is it a single numeric ID (0-76)?
- Is the format correct (just the number, no extra text)?
- Does it appear to be a reasonable classification for a banking query?

Good examples: "59", "0", "42" (single numbers)
Bad examples: "activate_my_card" (label instead of ID), "I think it's 59" (extra text), "" (empty)

Respond with JSON:
- "score": 0-10 (10 = perfect classification response)
  * 10: Valid ID (0-76), correct format
  * 5-9: Valid ID but could be better formatted
  * 1-4: Invalid format or out of range
  * 0: Empty or completely wrong
- "label": "excellent", "good", "needs_improvement", "poor"
- "reasoning": brief explanation

JSON response:""",
        # Analyze only assistant messages (not system or user)
        "filter_role": "assistant",
        "analyze_message_level": True,
        "analyze_conversation_level": False,
        # Inference configuration
        "inference_config": {
            "model_name": "gpt-4o-mini",  # or "gpt-4o", "claude-3-5-sonnet-20241022", etc.
            "engine": "openai",  # or "vllm", "native" for local models
            "temperature": 0.1,  # Low temperature for consistent judgments
            "max_tokens": 256,
            # For OpenAI, API key is read from OPENAI_API_KEY env var by default
            # For Anthropic, use ANTHROPIC_API_KEY env var
            # Or specify explicitly:
            # "api_key_env": "OPENAI_API_KEY",  # or "ANTHROPIC_API_KEY"
        },
        "batch_size": 10,  # Process 10 samples at a time
        "max_text_length": 4000,  # Truncate long texts
        "parse_json_response": True,  # Parse JSON from LLM response
    },
)


In [4]:
import os
from oumi.core.configs import AnalyzeConfig
from oumi.core.analyze.dataset_analyzer import DatasetAnalyzer

# Create the full AnalyzeConfig
config = AnalyzeConfig(
    # Dataset configuration
    dataset_path="/Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl",
    # Analysis configuration
    sample_count=5,  # Analyze first 100 samples (LLM calls can be expensive!)
    analyzers=[
        llm_response_quality_params,
        llm_response_quality_preset_params,
    ],  # Just the LLM judge
    # Output configuration
    output_path="./analysis_output/banking77_llm_judge",
    generate_report=False,  # Optional: generate HTML report
)

# Finalize and run
config.finalize_and_validate()

from oumi.core.analyze.dataset_analyzer import DatasetAnalyzer

analyzer = DatasetAnalyzer(config)
analyzer.analyze_dataset()


[2026-01-05 15:39:27,892][oumi][rank0][pid:16433][MainThread][INFO]][base_map_dataset.py:91] Creating map dataset (type: TextSftJsonLinesDataset)... dataset_name: 'custom'
[2026-01-05 15:39:27,893][oumi.utils.analysis_utils][rank0][pid:16433][MainThread][INFO]][analysis_utils.py:227] Loaded text dataset from: /Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl
[2026-01-05 15:39:27,894][oumi][rank0][pid:16433][MainThread][INFO]][dataset_analyzer.py:154] Loaded dataset from config: None
[2026-01-05 15:39:27,894][oumi][rank0][pid:16433][MainThread][INFO]][dataset_analyzer.py:320] Initialized sample analyzer: response_quality (type: llm_judge)
[2026-01-05 15:39:27,895][oumi][rank0][pid:16433][MainThread][INFO]][dataset_analyzer.py:320] Initialized sample analyzer: response_quality_preset (type: llm_judge)
[2026-01-05 15:39:27,896][oumi][rank0][pid:16433][MainThread][INFO]][dataset_analyzer.py:363] Starting analysis of dataset: None
[20

Converting Unknown Dataset to DataFrames: 100%|██████████| 5/5 [00:00<00:00, 1737.35item/s]


[2026-01-05 15:39:27,917][oumi.utils.analysis_utils][rank0][pid:16433][MainThread][INFO]][analysis_utils.py:1322] Adding default schema entries for 2 columns not in base schema: ['label', 'label_name']
[2026-01-05 15:39:27,918][oumi][rank0][pid:16433][MainThread][INFO]][llm_judge_analyzer.py:787] Skipping conversation-level analysis (analyze_conversation_level=False). Set analyze_conversation_level=True to enable.
[2026-01-05 15:39:27,918][oumi][rank0][pid:16433][MainThread][INFO]][llm_judge_analyzer.py:787] Skipping conversation-level analysis (analyze_conversation_level=False). Set analyze_conversation_level=True to enable.
[2026-01-05 15:39:27,919][oumi][rank0][pid:16433][MainThread][INFO]][llm_judge_analyzer.py:828] Evaluating 5 'assistant' messages (filtered from 15 total)
[2026-01-05 15:39:27,930][oumi][rank0][pid:16433][MainThread][INFO]][llm_judge_analyzer.py:444] Initialized LLM Judge with model: gpt-4o-mini, engine: openai


100%|██████████| 5/5 [00:09<00:00,  1.81s/it]


[2026-01-05 15:39:36,974][oumi][rank0][pid:16433][MainThread][INFO]][llm_judge_analyzer.py:828] Evaluating 5 'assistant' messages (filtered from 15 total)
[2026-01-05 15:39:36,976][oumi][rank0][pid:16433][MainThread][INFO]][llm_judge_analyzer.py:444] Initialized LLM Judge with model: gpt-4o-mini, engine: openai


100%|██████████| 5/5 [00:07<00:00,  1.48s/it]


In [5]:
schema = analyzer.get_schema()
print(f"len(schema): {len(schema)}")
merged_columns = analyzer.analysis_df.columns
print(f"len(merged_columns): {len(merged_columns)}")

len(schema): 18
len(merged_columns): 18


In [18]:
analyzer_names = [a.instance_id for a in config.analyzers]
print(f"analyzer_names: {analyzer_names}")

analyzer_names = [a.instance_id or a.id for a in config.analyzers]

print(f"analyzer_names: {analyzer_names}")


analyzer_names: ['response_quality', 'response_quality_preset']
analyzer_types: ['llm_judge', 'llm_judge']


# Conv level

In [19]:
from oumi.core.analyze.column_utils import (
    filter_analyzer_columns,
    get_analyzer_columns_by_analyzer,
    parse_analyzer_column_name,
)

conv_columns = analyzer.conversation_df.columns
row = analyzer.conversation_df.iloc[2]

# Choose the analzyer to analyze
analyzer_name = analyzer_names[0]


filtered_cols = filter_analyzer_columns(conv_columns, analyzer_id=analyzer_name)
if filtered_cols:
    print(f"Analyzer: {analyzer_name}")
    info = parse_analyzer_column_name(filtered_cols[0])
    # print("\nInput:")
    # print(f"source_column: {info.source_column}")
    print(f"{row[info.source_column]}\n")

    for col in filtered_cols:
        info = parse_analyzer_column_name(col)
        print(f"metric: {info.metric_name}")
        # print(f"type: {schema[col]['type']}")
        # print(f"content_type: {schema[col]['content_type']}")
        print(f"description: {schema[col]['description']}")
        print(f"value: {row[col]}")
        print("\n")
else:
    print(f"No columns found for analyzer: {analyzer_name}")


No columns found for analyzer: response_quality


# Message level

In [21]:
from oumi.core.analyze.column_utils import (
    filter_analyzer_columns,
    get_analyzer_columns_by_analyzer,
    parse_analyzer_column_name,
)

msg_columns = analyzer.message_df.columns
row = analyzer.message_df.iloc[5]

# Choose the analzyer to analyze
analyzer_name = analyzer_names[1]


filtered_cols = filter_analyzer_columns(msg_columns, analyzer_id=analyzer_name)
if filtered_cols:
    print(f"Analyzer: {analyzer_name}")
    info = parse_analyzer_column_name(filtered_cols[0])

    print("\nInput:")
    print(f"[{row['role']}]")
    # print(f"{row[info.source_column]}\n")
    # print(f"source_column: {info.source_column}")
    # print(f"{row[info.source_column]}\n")

    for col in filtered_cols:
        info = parse_analyzer_column_name(col)
        print(f"metric: {info.metric_name}")
        # print(f"type: {schema[col]['type']}")
        # print(f"content_type: {schema[col]['content_type']}")
        print(f"description: {schema[col]['description']}")
        print(f"value: {row[col]}")
        print("\n")
else:
    print(f"No columns found for analyzer: {analyzer_name}")


Analyzer: response_quality_preset

Input:
[assistant]
metric: score
description: LLM judge score (0-10, higher = better quality)
value: 1.0


metric: label
description: LLM judge label/category for the sample
value: poor


metric: reasoning
description: LLM judge reasoning/explanation
value: The response does not address the user's request, lacks detail, and is incomplete. It provides only a number without any context or explanation.


metric: raw_response
description: Raw LLM response before parsing
value: ```json
{
  "score": 1,
  "label": "poor",
  "reasoning": "The response does not address the user's request, lacks detail, and is incomplete. It provides only a number without any context or explanation."
}
```


